In [1]:
import pandas as pd
import numpy as np
import os as os
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, LassoCV, Lasso
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error

In [2]:
pitcher_csv = [file for file in os.listdir('./merged_data') if 'pitchers' in file]

In [3]:
li = []
for csv in pitcher_csv:
    li.append(pd.read_csv('./merged_data/' + csv))
pitchers = pd.concat(li, axis = 0, ignore_index = True)

In [4]:
# CSV of opposing teams average runs scored
runs = pd.read_csv('./Datasets/team_runs.csv')

In [5]:
# Formatting the Oppt column to merge opponents average runs
pitchers['Oppt'] = [opt.split()[-1].upper() for opt in pitchers['Oppt']]

In [7]:
# Merging in the opposing teams average runs(Later found to not be significant)
pitch_df = pd.merge(left = pitchers, right = runs, left_on = 'Oppt',
        right_on = 'team').drop(columns = ['team', 'Unnamed: 0', 'Tm Runs', 'Opp Runs'])

In [8]:
pitch_df.dropna(subset = ['DK pts','DK sal'])

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,num_L_bats,num_R_bats,...,MLB_ID,Starter,Bat order,DK posn,DK pts,DK sal,Team,Oppt,dblhdr,avg_runs
0,SF,6/11/2021,1,543101.0,Anthony DeSclafani,SP,N,NaN,5,3,...,543101,1.0,9,1.0,43.45,6400.0,SFO,WAS,NaN,3.95
1,SF,6/13/2021,1,456501.0,Johnny Cueto,SP,Y,NaN,3,5,...,456501,1.0,9,1.0,0.80,8100.0,SFO,WAS,NaN,3.95
2,ATL,6/3/2021,1,656353.0,Tucker Davidson,SP,Y,NaN,2,6,...,656353,1.0,9,1.0,19.15,4900.0,ATL,WAS,NaN,3.95
3,ATL,6/1/2021,1,608331.0,Max Fried,SP,Y,NaN,2,6,...,608331,1.0,9,1.0,6.85,7500.0,ATL,WAS,NaN,3.95
4,PIT,6/15/2021,1,542881.0,Tyler Anderson,SP,N,NaN,2,6,...,542881,1.0,9,1.0,2.90,7900.0,PIT,WAS,NaN,3.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,MIA,6/15/2021,1,669432.0,Trevor Rogers,SP,N,NaN,0,8,...,669432,1.0,9,1.0,21.10,10000.0,MIA,STL,NaN,3.91
260,MIA,6/14/2021,1,666129.0,Braxton Garrett,SP,N,NaN,0,8,...,666129,1.0,9,1.0,6.95,5000.0,MIA,STL,NaN,3.91
261,ARI,5/29/2021,1,571676.0,Seth Frankoff,SP,N,NaN,3,5,...,571676,1.0,9,1.0,-0.55,4800.0,ARI,STL,NaN,3.91
262,ARI,6/30/2021,1,642092.0,Riley Smith,SP,Y,NaN,3,5,...,642092,1.0,9,1.0,-3.45,7000.0,ARI,STL,NaN,3.91


In [9]:
pitch_df.columns

Index(['team code', ' game_date', ' game_number', ' mlb id', 'Name',
       ' batting order', ' confirmed', ' position', 'num_L_bats', 'num_R_bats',
       'is_nl', 'k_9_l', 'bb_9_l', 'k_bb_l', 'hr_9_l', 'k_pct_l', 'bb_pct_l',
       'k_bb_pct_l', 'bat_avg_l', 'whip_l', 'babip_l', 'lob_pct_l', 'fip_l',
       'xfip_l', 'k_9_r', 'bb_9_r', 'k_bb_r', 'hr_9_r', 'k_pct_r', 'bb_pct_r',
       'k_bb_pct_r', 'bat_avg_r', 'whip_r', 'babip_r', 'lob_pct_r', 'fip_r',
       'xfip_r', 'k_9_l_tot', 'bb_9_l_tot', 'k_bb_l_tot', 'hr_9_l_tot',
       'k_pct_l_tot', 'bb_pct_l_tot', 'k_bb_pct_l_tot', 'bat_avg_l_tot',
       'whip_l_tot', 'babip_l_tot', 'lob_pct_l_tot', 'fip_l_tot', 'xfip_l_tot',
       'k_9_r_tot', 'bb_9_r_tot', 'k_bb_r_tot', 'hr_9_r_tot', 'k_pct_r_tot',
       'bb_pct_r_tot', 'k_bb_pct_r_tot', 'bat_avg_r_tot', 'whip_r_tot',
       'babip_r_tot', 'lob_pct_r_tot', 'fip_r_tot', 'xfip_r_tot', 'Date',
       'GID', 'MLB_ID', 'Starter', 'Bat order', 'DK posn', 'DK pts', 'DK sal',
       'Team'

In [22]:
# Selecting features to be exported to google colab
# The features will be trimmed down further in google colab
p_feat = ['num_L_bats', 'num_R_bats',
       'is_nl', 'k_9_l', 'bb_9_l', 'k_bb_l', 'hr_9_l', 'k_pct_l', 'bb_pct_l',
       'k_bb_pct_l', 'bat_avg_l', 'whip_l', 'babip_l', 'lob_pct_l', 'fip_l',
       'xfip_l', 'k_9_r', 'bb_9_r', 'k_bb_r', 'hr_9_r', 'k_pct_r', 'bb_pct_r',
       'k_bb_pct_r', 'bat_avg_r', 'whip_r', 'babip_r', 'lob_pct_r', 'fip_r',
       'xfip_r', 'k_9_l_tot', 'bb_9_l_tot', 'k_bb_l_tot', 'hr_9_l_tot',
       'k_pct_l_tot', 'bb_pct_l_tot', 'k_bb_pct_l_tot', 'bat_avg_l_tot',
       'whip_l_tot', 'babip_l_tot', 'lob_pct_l_tot', 'fip_l_tot', 'xfip_l_tot',
       'k_9_r_tot', 'bb_9_r_tot', 'k_bb_r_tot', 'hr_9_r_tot', 'k_pct_r_tot',
       'bb_pct_r_tot', 'k_bb_pct_r_tot', 'bat_avg_r_tot', 'whip_r_tot',
       'babip_r_tot', 'lob_pct_r_tot', 'fip_r_tot', 'xfip_r_tot', 'DK sal']
X = pitch_df[p_feat]
y = pitch_df['DK pts']

In [23]:
# preparing to export data to use in neural network in google colab
p_feat.append('DK pts')
final_pitch = pitch_df[p_feat]
final_pitch.head()

,num_L_bats,num_R_bats,is_nl,k_9_l,bb_9_l,k_bb_l,hr_9_l,k_pct_l,bb_pct_l,k_bb_pct_l,...,bb_pct_r_tot,k_bb_pct_r_tot,bat_avg_r_tot,whip_r_tot,babip_r_tot,lob_pct_r_tot,fip_r_tot,xfip_r_tot,DK sal,DK pts
0,5,3,1,7.662162,3.283784,2.333333,2.006757,0.189189,0.081081,0.108108,...,0.260870,0.333333,0.600000,3.235294,0.692308,2.380952,11.046368,13.590725,6400.0,43.45
1,3,5,1,7.867550,3.754967,2.095238,0.715232,0.193833,0.092511,0.101322,...,0.239130,0.804348,1.232228,5.494186,1.406250,3.146853,19.335912,20.025662,8100.0,0.80
2,2,6,1,18.000000,9.000000,2.000000,0.000000,0.470588,0.235294,0.235294,...,0.857143,-0.285714,1.371429,9.000000,1.200000,2.678571,41.991407,40.334653,4900.0,19.15
3,2,6,1,7.105263,3.789474,1.875000,0.000000,0.187500,0.100000,0.087500,...,0.504673,0.915888,1.468531,7.691630,1.864078,4.883721,20.995703,22.967491,7500.0,6.85
4,2,6,1,6.402062,2.226804,2.875000,0.835052,0.167883,0.058394,0.109489,...,0.538259,0.680739,1.495496,7.887640,1.670588,4.387352,26.192616,30.090019,7900.0,2.90


In [24]:
final_pitch.to_csv('./merged_data/final_pitch.csv', index = False)